In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from pathlib import Path

# --- 경로 설정 ---
try:
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
except NameError:
    PROJECT_ROOT = Path.cwd().parent
DATA_DIR = PROJECT_ROOT / "Data"
RESULTS_DIR = PROJECT_ROOT / "Results"
RAW_DATA_PATH = DATA_DIR / "Raw data" / "downloaded_stock_prices.csv"
OPTIMAL_WEIGHTS_SAVE_PATH = RESULTS_DIR / "optimized_weights_all_periods.xlsx"

# --- 종목 및 기간 설정 ---
STOCKS = [
    'SPY', 'QQQ', 'EFA', 'EEM', 'SSO', 'TQQQ', # 주식
    'TLT', 'IEF', 'SHY',       # 채권
    'GLD', 'DBC',              # 원자재
    'BTC-USD', 'ETH-USD', 'XRP-USD'                  # 암호화폐
]
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=10*365)

# --- 데이터 처리 함수 ---
def get_data(tickers, start, end, save_local_path=None):
    """지정된 티커의 주가 데이터를 다운로드하고, 'Adj Close'를 사용하며, 컬럼 순서를 재정렬합니다."""
    print(f"'{tickers}' 종목 데이터 다운로드 시도 중...")
    data = yf.download(tickers, start=start, end=end, auto_adjust=False)
    if data.empty: raise ValueError("데이터 다운로드 실패")
    
    price_data = data['Adj Close'].dropna()
    present_tickers = [ticker for ticker in tickers if ticker in price_data.columns]
    price_data = price_data[present_tickers]
    
    if save_local_path:
        os.makedirs(os.path.dirname(save_local_path), exist_ok=True)
        price_data.to_csv(save_local_path)
        print(f"수정 종가 데이터가 로컬에 저장되었습니다: {save_local_path}")
    return price_data

def calculate_returns(data, period='daily'):
    """주가 데이터를 기반으로 수익률을 계산합니다."""
    if period == 'weekly': return data.resample('W-FRI').last().pct_change().dropna()
    if period == 'monthly': return data.resample('M').last().pct_change().dropna()
    return data.pct_change().dropna()

# --- 포트폴리오 성과 지표 계산 함수들 ---
def portfolio_metrics(weights, returns, period_type='daily'):
    """포트폴리오의 주요 성과 지표(수익률, 변동성, 샤프지수, 칼마지수)를 계산합니다."""
    periods_per_year = {'daily': 252, 'weekly': 52, 'monthly': 12}
    annualizer = periods_per_year.get(period_type, 252)

    portfolio_returns = (returns * weights).sum(axis=1)
    
    annual_return = portfolio_returns.mean() * annualizer
    annual_volatility = portfolio_returns.std() * np.sqrt(annualizer)
    
    sharpe_ratio = annual_return / annual_volatility if annual_volatility > 0 else 0
    
    # Calmar Ratio 계산
    cumulative_returns = (1 + portfolio_returns).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    max_drawdown = drawdown.min()
    
    # CAGR 계산 (기간이 0일 경우 방지)
    num_days = len(cumulative_returns)
    if num_days == 0: return 0, 0, 0, 0
    cagr = (cumulative_returns.iloc[-1])**(annualizer/num_days) - 1

    # Calmar Ratio 계산 (MDD가 0일 경우 방지)
    epsilon = 1e-10 # 0으로 나누는 것을 방지하기 위한 아주 작은 값
    calmar_ratio = cagr / (abs(max_drawdown) + epsilon)
    
    return annual_return, annual_volatility, sharpe_ratio, calmar_ratio

# --- 최적화 목표 함수들 (최소화 대상) ---
def neg_sharpe_ratio(weights, returns, period_type):
    return -portfolio_metrics(weights, returns, period_type)[2]

def portfolio_volatility(weights, returns, period_type):
    return portfolio_metrics(weights, returns, period_type)[1]

def neg_calmar_ratio(weights, returns, period_type):
    """Calmar Ratio 최대화를 위해 음수 값을 반환합니다."""
    # Calmar Ratio는 일별 수익률 기반으로 계산하는 것이 가장 정확합니다.
    return -portfolio_metrics(weights, returns, 'daily')[3]

# --- 최적화 실행 함수들 (전략별) ---
def optimize_portfolio(objective_func, returns, period_type, constraints, bounds, init_guess):
    """범용 최적화 실행 함수"""
    result = minimize(objective_func, init_guess, args=(returns, period_type),
                      method='SLSQP', bounds=bounds, constraints=constraints)
    return result.x

def calculate_risk_parity_weights(returns):
    """위험 패리티 포트폴리오의 가중치를 계산합니다."""
    vols = returns.std()
    inv_vols = 1.0 / vols
    return (inv_vols / inv_vols.sum()).values

# --- 메인 실행 흐름 ---
if __name__ == "__main__":
    stock_data = get_data(STOCKS, START_DATE, END_DATE, save_local_path=RAW_DATA_PATH)
    
    if not stock_data.empty:
        results = {}
        final_tickers = stock_data.columns.tolist()
        num_assets = len(final_tickers)
        
        # 공통 최적화 설정
        init_guess = np.array([1/num_assets] * num_assets)
        bounds = tuple((0, 1) for _ in range(num_assets))
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}]
        
        # --- 실행할 전략 목록 ---
        STRATEGIES = {
            # 새로운 핵심 전략: Calmar Ratio 최대화
            'max_calmar': {'func': neg_calmar_ratio, 'args': {'period_type': 'daily'}},
            
            # 기타 전략들
            'risk_parity': {'func': calculate_risk_parity_weights, 'is_optimizer': False},
            'min_variance': {'func': portfolio_volatility, 'args': {'period_type': 'daily'}},
            'daily_30_cap': {'func': neg_sharpe_ratio, 'args': {'period_type': 'daily'}, 'bounds': tuple((0, 0.3) for _ in range(num_assets))},
            'target_return_18': {'func': portfolio_volatility, 'args': {'period_type': 'daily'}, 'constraints': [
                {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
                {'type': 'ineq', 'fun': lambda w: portfolio_metrics(w, calculate_returns(stock_data), 'daily')[0] - 0.18}
            ]},

            # --- 기존 샤프지수 모델 (주석 처리) ---
            # 'daily_max_sharpe': {'func': neg_sharpe_ratio, 'args': {'period_type': 'daily'}},
            # 'weekly_max_sharpe': {'func': neg_sharpe_ratio, 'args': {'period_type': 'weekly'}},
            # 'monthly_max_sharpe': {'func': neg_sharpe_ratio, 'args': {'period_type': 'monthly'}},
        }

        for name, params in STRATEGIES.items():
            print(f"\n--- '{name}' 모델 분석 시작 ---")
            
            # Risk Parity는 최적화 함수가 아니므로 별도 처리
            if params.get('is_optimizer') is False:
                weights = params['func'](calculate_returns(stock_data))
            else:
                period = params.get('args', {}).get('period_type', 'daily')
                returns = calculate_returns(stock_data, period=period)
                
                # 전략별 제약조건 및 바운드 설정
                b = params.get('bounds', bounds)
                c = params.get('constraints', constraints)

                weights = optimize_portfolio(params['func'], returns, **params['args'],
                                             constraints=c, bounds=b, init_guess=init_guess)
            
            results[name] = {'weights': weights}
            print(f"최적 가중치: " + ", ".join([f"{s}: {w*100:.2f}%" for s, w in zip(final_tickers, weights)]))

        # --- 모든 모델의 가중치를 하나의 Excel 파일에 저장 ---
        if results:
            os.makedirs(RESULTS_DIR, exist_ok=True)
            with pd.ExcelWriter(OPTIMAL_WEIGHTS_SAVE_PATH, engine='openpyxl') as writer:
                for model_name, data in results.items():
                    weights_df = pd.DataFrame({'Ticker': final_tickers, 'Optimal_Weight': data['weights']})
                    weights_df.to_excel(writer, sheet_name=f'{model_name}_weights', index=False)
            print(f"\n모든 모델의 최적 가중치가 저장되었습니다: {OPTIMAL_WEIGHTS_SAVE_PATH}")


'['SPY', 'QQQ', 'EFA', 'EEM', 'SSO', 'TQQQ', 'TLT', 'IEF', 'SHY', 'GLD', 'DBC', 'BTC-USD', 'ETH-USD', 'XRP-USD']' 종목 데이터 다운로드 시도 중...



*********************100%***********************]  14 of 14 completed

수정 종가 데이터가 로컬에 저장되었습니다: C:\Users\012oov\Documents\012\Quant\1. MPT_Back\Data\Raw data\downloaded_stock_prices.csv

--- 'max_calmar' 모델 분석 시작 ---
최적 가중치: SPY: 2.04%, QQQ: 22.02%, EFA: 0.00%, EEM: 0.00%, SSO: 0.00%, TQQQ: 1.30%, TLT: 0.00%, IEF: 0.00%, SHY: 0.00%, GLD: 48.80%, DBC: 16.69%, BTC-USD: 0.00%, ETH-USD: 0.00%, XRP-USD: 9.15%

--- 'risk_parity' 모델 분석 시작 ---
최적 가중치: SPY: 4.28%, QQQ: 3.46%, EFA: 4.64%, EEM: 3.98%, SSO: 2.14%, TQQQ: 1.17%, TLT: 5.27%, IEF: 11.87%, SHY: 49.93%, GLD: 5.75%, DBC: 4.60%, BTC-USD: 1.24%, ETH-USD: 0.96%, XRP-USD: 0.72%

--- 'min_variance' 모델 분석 시작 ---
최적 가중치: SPY: 0.81%, QQQ: 0.00%, EFA: 0.00%, EEM: 0.00%, SSO: 0.00%, TQQQ: 0.00%, TLT: 0.00%, IEF: 0.00%, SHY: 98.02%, GLD: 0.00%, DBC: 1.17%, BTC-USD: 0.00%, ETH-USD: 0.00%, XRP-USD: 0.00%

--- 'daily_30_cap' 모델 분석 시작 ---
최적 가중치: SPY: 0.00%, QQQ: 17.56%, EFA: 0.00%, EEM: 0.00%, SSO: 0.00%, TQQQ: 0.00%, TLT: 0.00%, IEF: 12.46%, SHY: 30.00%, GLD: 30.00%, DBC: 2.67%, BTC-USD: 4.52%, ETH-USD: 0.00%, XRP-USD: 2